In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45990
2,Huelva,Confirmados PDIA 14 días,320
3,Huelva,Tasa PDIA 14 días,"62,357503361459166"
4,Huelva,Confirmados PDIA 7 días,188
5,Huelva,Tasa PDIA 7 dias,"36,63503322485726"
6,Huelva,Total Confirmados,46193
7,Huelva,Curados,44998
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45990.0


/tmp/ipykernel_4742/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12648.0


/tmp/ipykernel_4742/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4742/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4742/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4742/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3345 personas en los últimos 7 días 

Un positivo PCR cada 2115 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45990.0,188.0,320.0,513170.0,36.635033,62.357503,95.0
Huelva-Costa,27203.0,112.0,183.0,289548.0,38.680979,63.201956,55.0
Condado-Campiña,14285.0,43.0,91.0,156231.0,27.523347,58.247083,27.0
Huelva (capital),12648.0,43.0,68.0,143837.0,29.894951,47.275736,25.0
Sierra de Huelva-Andévalo Central,4075.0,32.0,45.0,67391.0,47.484085,66.774495,12.0
Campofrío,45.0,22.0,23.0,713.0,3085.553997,3225.806452,10.0
Cartaya,2088.0,31.0,57.0,20083.0,154.359408,283.822138,9.0
Almonte,2318.0,8.0,16.0,24507.0,32.643734,65.287469,8.0
Lepe,3043.0,12.0,18.0,27880.0,43.041607,64.562410,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,45.0,22.0,23.0,713.0,3085.553997,3225.806452,10.0
Granado (El),43.0,3.0,3.0,532.0,563.909774,563.909774,NaN
Zalamea la Real,123.0,4.0,9.0,3054.0,130.975769,294.695481,1.0
Cartaya,2088.0,31.0,57.0,20083.0,154.359408,283.822138,9.0
Palos de la Frontera,1060.0,9.0,17.0,11742.0,76.647931,144.779424,6.0
Moguer,2006.0,12.0,25.0,21867.0,54.877212,114.327525,5.0
Palma del Condado (La),1431.0,4.0,9.0,10801.0,37.033608,83.325618,1.0
Isla Cristina,3152.0,9.0,17.0,21393.0,42.069836,79.465246,6.0
Minas de Riotinto,181.0,2.0,3.0,3812.0,52.465897,78.698846,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,1296.0,2.0,7.0,14387.0,13.901439,48.655036,1.0,0.285714
Punta Umbría,1277.0,2.0,5.0,15355.0,13.025073,32.562683,1.0,0.400000
Zalamea la Real,123.0,4.0,9.0,3054.0,130.975769,294.695481,1.0,0.444444
Palma del Condado (La),1431.0,4.0,9.0,10801.0,37.033608,83.325618,1.0,0.444444
Condado-Campiña,14285.0,43.0,91.0,156231.0,27.523347,58.247083,27.0,0.472527
Moguer,2006.0,12.0,25.0,21867.0,54.877212,114.327525,5.0,0.480000
Almonte,2318.0,8.0,16.0,24507.0,32.643734,65.287469,8.0,0.500000
San Juan del Puerto,762.0,1.0,2.0,9411.0,10.625863,21.251727,0.0,0.500000
Aracena,672.0,1.0,2.0,8255.0,12.113870,24.227741,0.0,0.500000
